# Apache Parquet

* is a `columnar data format`
* `independent of data processing framework, data model or programming language`
* uses `record shredding and assembly` algorithm
* supports various `compression and encoding` schemes
* parquet format supports `nested data`

## Parquet file structure

* A parquet file consists of one or more `row groups`
* A row group contains exactly one `one column chunk` per column
* A column chunk contains one or more pages

## Metadata

* File metadata
* Column Chunk metadata
* page header metadata

## Data type

* The types supported by the parquet file format are minimal, how the types effect disk storage?
* BOOLEAN: 1 bit boolean
* INT32: 32 bit signed ints
* INT64: 64 bit signed ints
* INT96: 96 bit signed ints
* FLOAT: IEEE 32-bit floating point values
* DOUBLE: IEEE 64-bit floating point values
* BYTE_ARRAY: arbitrarily long byte arrays.

## Key Notes
* 2 libraries that support writing parquet files - **pyarrow** and **fastparquet**
* Both of them don't support nested data
    * [fastparquet nested data issue](https://github.com/dask/fastparquet/issues/272)
    * [pyarrow nested data issue](https://issues.apache.org/jira/browse/ARROW-1644?src=confmacro)
* **pyspark.sql** uses py4j which runs on JVM to generate nested data. [Reference](https://stackoverflow.com/questions/32940416/methods-for-writing-parquet-files-using-python)

## Using pyspark to Write/Read parquet

## Setup pyspark python

In [1]:
%%sh
export PYSPARK_PYTHON=$(which python3)
export PYSPARK_DRIVER_PYTHON=$(which python3)

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
# create spark session
spark = SparkSession \
    .builder \
    .appName("Pyspark Parquet") \
    .getOrCreate()
# create spark context
sc = spark.sparkContext

## Setup sample flattened data

In [3]:
import random
import uuid
import json

def gen_sample_acct():
    return dict(acct_id=random.randint(1,99999999),acct_feature1=str(uuid.uuid4()))

def gen_sample_app():
    return dict(app_id=random.randint(1,99999999),app_feature1=str(uuid.uuid4()))

def gen_sample_accounts(num_records):
    accts = [gen_sample_acct() for i in range(num_records)]
    return accts

def gen_sample_apps(num_records):
    apps = [gen_sample_app() for i in range(num_records)]
    return apps

def gen_sample_customers(num_records):
    customers = [dict(cust_id=random.randint(1,99999999),
                      cust_feature1=str(uuid.uuid4()),
                      accts=gen_sample_accounts(num_records),
                      apps=gen_sample_apps(num_records)
                      ) for i in range(num_records)]
    return customers

def write_json(sample_dict, filename):
    with open(filename, 'w') as f:
        f.write(json.dumps(sample_dict))
    return "success"

In [4]:
%%sh 
ls -lt | grep -i "sample_data"

-rw-r--r--  1 sharattadimalla  staff  16787 Nov 10 12:33 sample_data.json
drwxr-xr-x  6 sharattadimalla  staff    192 Nov 10 11:43 sample_data.parquet


In [5]:
sample_accts = gen_sample_accounts(5)
write_json(sample_accts, 'sample_accounts.json')

'success'

In [7]:
%%sh 
ls -lt | grep -i "sample_"

-rw-r--r--  1 sharattadimalla  staff    399 Nov 10 12:40 sample_accounts.json
drwxr-xr-x  6 sharattadimalla  staff    192 Nov 10 12:36 sample_customer.parquet
-rw-r--r--  1 sharattadimalla  staff  16787 Nov 10 12:35 sample_customers.json
-rw-r--r--  1 sharattadimalla  staff  16787 Nov 10 12:33 sample_data.json
drwxr-xr-x  6 sharattadimalla  staff    192 Nov 10 11:43 sample_data.parquet


## pyspark - Read json data and write parquet with flat structure

In [10]:
df = spark.read.json('sample_accounts.json')

In [11]:
df.printSchema()

root
 |-- acct_feature1: string (nullable = true)
 |-- acct_id: long (nullable = true)



In [12]:
df.show(2)

+--------------------+--------+
|       acct_feature1| acct_id|
+--------------------+--------+
|e4986ed3-2fd8-4ab...|53294007|
|64494600-776e-40a...| 6944675|
+--------------------+--------+
only showing top 2 rows



In [24]:
df.write.parquet('sample_accounts.parquet',mode='overwrite')

In [14]:
%%sh
ls -lt | grep -i "sample_"

drwxr-xr-x  6 sharattadimalla  staff    192 Nov 10 12:42 sample_accounts.parquet
-rw-r--r--  1 sharattadimalla  staff    399 Nov 10 12:40 sample_accounts.json
drwxr-xr-x  6 sharattadimalla  staff    192 Nov 10 12:36 sample_customer.parquet
-rw-r--r--  1 sharattadimalla  staff  16787 Nov 10 12:35 sample_customers.json
-rw-r--r--  1 sharattadimalla  staff  16787 Nov 10 12:33 sample_data.json
drwxr-xr-x  6 sharattadimalla  staff    192 Nov 10 11:43 sample_data.parquet


In [15]:
df2 = spark.read.parquet('sample_accounts.parquet')

In [16]:
df2.printSchema()

root
 |-- acct_feature1: string (nullable = true)
 |-- acct_id: long (nullable = true)



## pyspark - write nested data

In [17]:
sample_customers = gen_sample_customers(10)
sample_customers

[{'cust_id': 24393450,
  'cust_feature1': 'fc3dc303-cb35-4e24-a769-f7db16f35d5e',
  'accts': [{'acct_id': 91715491,
    'acct_feature1': '918f42dd-cdf4-4092-ae5d-1ad7c01b65a0'},
   {'acct_id': 21356257,
    'acct_feature1': '394d628f-2e3f-489d-aa85-54d10eeae063'},
   {'acct_id': 84133290,
    'acct_feature1': '8dcde694-9151-4644-a9b3-19d32d40e38e'},
   {'acct_id': 70436822,
    'acct_feature1': '35576bd7-a191-4ef1-b3a8-0e8318f62d8e'},
   {'acct_id': 15791866,
    'acct_feature1': '75c66f37-272f-4af5-90aa-683a3dc976ae'},
   {'acct_id': 53113259,
    'acct_feature1': '45771b29-be2e-49a6-96ba-a0f3acf0fc16'},
   {'acct_id': 2201428,
    'acct_feature1': '58e02aa1-1ac7-4cbc-b872-fa56d12853f7'},
   {'acct_id': 70063115,
    'acct_feature1': '28868f83-1d7c-40d8-9705-0d61c27268e4'},
   {'acct_id': 43814088,
    'acct_feature1': '22eb2e11-372c-41db-8046-97258570229a'},
   {'acct_id': 4526149,
    'acct_feature1': '82524507-c1c0-4638-bf83-fc181118c5d6'}],
  'apps': [{'app_id': 63868287,
    'app

In [18]:
write_json(sample_customers, 'sample_customers.json')

'success'

In [19]:
df4 = spark.read.json('sample_customers.json')

In [20]:
df4.printSchema()

root
 |-- accts: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- acct_feature1: string (nullable = true)
 |    |    |-- acct_id: long (nullable = true)
 |-- apps: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- app_feature1: string (nullable = true)
 |    |    |-- app_id: long (nullable = true)
 |-- cust_feature1: string (nullable = true)
 |-- cust_id: long (nullable = true)



In [21]:
df4.show(5)

+--------------------+--------------------+--------------------+--------+
|               accts|                apps|       cust_feature1| cust_id|
+--------------------+--------------------+--------------------+--------+
|[[918f42dd-cdf4-4...|[[5759c19d-7429-4...|fc3dc303-cb35-4e2...|24393450|
|[[e2d3283d-d014-4...|[[5f469988-95f1-4...|afb22f3d-e0c9-480...|30060538|
|[[5a6bacbf-6624-4...|[[78974592-8cf5-4...|8abe0a68-42bd-41a...|33580110|
|[[381e43b0-cbe3-4...|[[cda4d8bb-ef13-4...|b6bef384-20ff-425...| 1570154|
|[[c0478f8d-2c45-4...|[[78c69004-d257-4...|bc7567fd-5fbd-490...|48004070|
+--------------------+--------------------+--------------------+--------+
only showing top 5 rows



In [23]:
df4.write.parquet('sample_customer.parquet', mode='overwrite')

In [25]:
%%sh
ls -lt | grep -i "sample_customer"

drwxr-xr-x  6 sharattadimalla  staff    192 Nov 10 12:43 sample_customer.parquet
-rw-r--r--  1 sharattadimalla  staff  16784 Nov 10 12:42 sample_customers.json


In [26]:
df5 = spark.read.parquet('sample_customer.parquet')

In [27]:
df5.printSchema()

root
 |-- accts: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- acct_feature1: string (nullable = true)
 |    |    |-- acct_id: long (nullable = true)
 |-- apps: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- app_feature1: string (nullable = true)
 |    |    |-- app_id: long (nullable = true)
 |-- cust_feature1: string (nullable = true)
 |-- cust_id: long (nullable = true)



In [28]:
df5.show(5)

+--------------------+--------------------+--------------------+--------+
|               accts|                apps|       cust_feature1| cust_id|
+--------------------+--------------------+--------------------+--------+
|[[918f42dd-cdf4-4...|[[5759c19d-7429-4...|fc3dc303-cb35-4e2...|24393450|
|[[e2d3283d-d014-4...|[[5f469988-95f1-4...|afb22f3d-e0c9-480...|30060538|
|[[5a6bacbf-6624-4...|[[78974592-8cf5-4...|8abe0a68-42bd-41a...|33580110|
|[[381e43b0-cbe3-4...|[[cda4d8bb-ef13-4...|b6bef384-20ff-425...| 1570154|
|[[c0478f8d-2c45-4...|[[78c69004-d257-4...|bc7567fd-5fbd-490...|48004070|
+--------------------+--------------------+--------------------+--------+
only showing top 5 rows

